In [1]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer, util
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer

2023-05-27 13:24:07.204479: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
school_final = pd.read_csv('school_final.csv')
school_final = school_final.drop('Unnamed: 0', axis=1)
school_final.head()

,Title,Author,publisher,description,published_date,categories,Type of Ban,State,District,Date of Challenge/Removal,Origin of Challenge,State_Count
0,Ace of Spades,"Àbíké-Íyímídé, Faridah",Feiwel & Friends,"Gossip Girl meets Get Out in Ace of Spades, a ...",2021-06-01,['Young Adult Fiction'],Banned in Libraries and Classrooms,Florida,Indian River County School District,November 2021,Administrator,923
1,Clap When You Land,"Acevedo, Elizabeth",HarperCollins,In a novel-in-verse that brims with grief and ...,2020-05-05,['Young Adult Fiction'],Banned in Classrooms,Pennsylvania,Central York School District,August 2021,Administrator,485
2,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned in Libraries,Florida,Indian River County School District,November 2021,Administrator,923
3,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned in Libraries and Classrooms,New York,Marlboro Central School District,February 2022,Administrator,15
4,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned Pending Investigation,Texas,Fredericksburg Independent School District,March 2022,Administrator,1239


In [13]:
school_final_copy = school_final.copy()

In [14]:
school_final_copy.drop_duplicates(subset='Title', keep='first', inplace=True)
school_final_copy.head()

,Title,Author,publisher,description,published_date,categories,Type of Ban,State,District,Date of Challenge/Removal,Origin of Challenge,State_Count
0,Ace of Spades,"Àbíké-Íyímídé, Faridah",Feiwel & Friends,"Gossip Girl meets Get Out in Ace of Spades, a ...",2021-06-01,['Young Adult Fiction'],Banned in Libraries and Classrooms,Florida,Indian River County School District,November 2021,Administrator,923
1,Clap When You Land,"Acevedo, Elizabeth",HarperCollins,In a novel-in-verse that brims with grief and ...,2020-05-05,['Young Adult Fiction'],Banned in Classrooms,Pennsylvania,Central York School District,August 2021,Administrator,485
2,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned in Libraries,Florida,Indian River County School District,November 2021,Administrator,923
6,Call Me By Your Name (Call Me By Your Name Ser...,"Aciman, André",NaN,"The sultry, sensual novel of obsession and des...",2020-06-02,['Gay teenagers'],Banned Pending Investigation,Virginia,Spotsylvania County Public Schools,November 2021,Administrator,30
7,"How I Paid for College: A Novel of Sex, Theft,...","Acito, Marc",Bloomsbury Publishing,'A comic coming-of-age novel ... it's an exube...,2009-08-17,['Fiction'],Banned Pending Investigation,Florida,Indian River County School District,November 2021,Administrator,923


In [6]:
# convert to lowercase and remove words for modeling
school_final_copy['description'] = school_final_copy['description'].str.lower()

words_to_remove = ["she's", "he's", "shes", "hes", "bestselling", "bestseller", 'new', 'york', 'times', 'book','author']

school_final_copy['description'] = school_final_copy['description'].apply(
    lambda text: ' '.join([word for word in text.split() if word not in words_to_remove])
)

In [7]:
docs = school_final_copy.description.values

In [8]:
vectorizer_model = CountVectorizer(ngram_range=(1, 3), stop_words='english')
#umap_model = UMAP(random_state=101)

topic_model = BERTopic(
    #umap_model=umap_model,
    vectorizer_model = vectorizer_model,
    #nr_topics = 'auto',
    #min_topic_size = 4,
    language='english', 
    calculate_probabilities=True,
    verbose=True
)
topics, probs = topic_model.fit_transform(docs)

topic_model.get_topic_info()

Batches:   0%|          | 0/73 [00:00<?, ?it/s]

2023-05-27 13:28:45,288 - BERTopic - Transformed documents to Embeddings
2023-05-27 13:28:59,439 - BERTopic - Reduced dimensionality
2023-05-27 13:28:59,722 - BERTopic - Clustered reduced embeddings


,Topic,Count,Name
0,-1,1126,-1_story_life_love_family
1,0,105,0_school_boy_gay_alex
2,1,98,1_love_friends_best_school
3,2,89,2_fantasy_series_war_ash
4,3,88,3_black_white_civil_history
5,4,64,4_art_paintings_works_artists
6,5,52,5_gender_transgender_lgbt_people
7,6,40,6_women_space_science_world
8,7,38,7_students_schools_teaching_education
9,8,34,8_killer_theo_police_murder


In [9]:
# find best strategy for removing outliers by looking at results

#new_topics = topic_model.reduce_outliers(docs, topics, probabilities=probs, strategy="probabilities")
#new_topics = topic_model.reduce_outliers(docs, topics, strategy="distributions")
new_topics = topic_model.reduce_outliers(docs, topics, strategy="c-tf-idf")
#new_topics = topic_model.reduce_outliers(docs, topics, strategy="embeddings")

# updates topic representation
#topic_model.update_topics(docs, topics=new_topics, vectorizer_model=vectorizer_model)

# updates topic frequency only without changing overall representation
documents = pd.DataFrame({"Document": docs, "Topic": new_topics})
topic_model._update_topic_size(documents)
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,1,-1_story_life_love_family
1,0,437,0_school_boy_gay_alex
2,1,329,1_love_friends_best_school
3,2,179,2_fantasy_series_war_ash
4,3,203,3_black_white_civil_history
5,4,88,4_art_paintings_works_artists
6,5,94,5_gender_transgender_lgbt_people
7,6,57,6_women_space_science_world
8,7,50,7_students_schools_teaching_education
9,8,53,8_killer_theo_police_murder


In [15]:
# merge topics to corresponding descriptions in original dataframe

# Step 1: Get topic information from the topic model and exclude outliers
topic_df = topic_model.get_topic_info()[1:]
#topic_df = topic_model.get_topic_info()

# Step 2: Create a DataFrame with descriptions and corresponding topics
topics_df = pd.DataFrame({"description": docs, "Topic": topic_model.topics_})

# Step 3: Merge topics DataFrame with topic information DataFrame
merged_df = topics_df.merge(topic_df, on='Topic', how='left')

# Step 4: Merge copied DataFrame with merged_df using concat
merged_s = pd.concat([school_final_copy.reset_index(drop=True), merged_df['Name'].reset_index(drop=True)], axis=1)

# Step 5: Remove the number and underscore from the 'Name' column
merged_s['Name'] = merged_s['Name'].str.replace(r'^\d+_', '', regex=True)

# Perform a merge based on matching values in the 'Title' column
merged_df1 = school_final.merge(merged_s[['Title', 'Name']], on='Title', how='left')

# Add the 'Name' column to the 'school_final' DataFrame and fill it with the corresponding values
school_final['Name'] = merged_df1['Name']
school_final.rename(columns={'Name': 'Topic'}, inplace=True)
school_final.head()

,Title,Author,publisher,description,published_date,categories,Type of Ban,State,District,Date of Challenge/Removal,Origin of Challenge,State_Count,Topic
0,Ace of Spades,"Àbíké-Íyímídé, Faridah",Feiwel & Friends,"Gossip Girl meets Get Out in Ace of Spades, a ...",2021-06-01,['Young Adult Fiction'],Banned in Libraries and Classrooms,Florida,Indian River County School District,November 2021,Administrator,923,love_friends_best_school
1,Clap When You Land,"Acevedo, Elizabeth",HarperCollins,In a novel-in-verse that brims with grief and ...,2020-05-05,['Young Adult Fiction'],Banned in Classrooms,Pennsylvania,Central York School District,August 2021,Administrator,485,love_friends_best_school
2,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned in Libraries,Florida,Indian River County School District,November 2021,Administrator,923,black_woodson_poems_poetry
3,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned in Libraries and Classrooms,New York,Marlboro Central School District,February 2022,Administrator,15,black_woodson_poems_poetry
4,The Poet X,"Acevedo, Elizabeth",HarperCollins,A National Book Award Longlist title! Fans of ...,2019-03-19,['Young Adult Fiction'],Banned Pending Investigation,Texas,Fredericksburg Independent School District,March 2022,Administrator,1239,black_woodson_poems_poetry


In [25]:
school_final['Topic'].value_counts()

Topic
school_boy_gay_alex                                                    875
love_friends_best_school                                               599
black_white_civil_history                                              322
fantasy_series_war_ash                                                 278
gender_transgender_lgbt_people                                         255
sex_sexual_information_pregnancy                                       116
killer_theo_police_murder                                              113
nazi_story_vonnegut_vonneguts                                          110
art_paintings_works_artists                                            103
love_annie_story_classic                                                93
families_family_red_2019                                                86
black_woodson_poems_poetry                                              86
women_space_science_world                                               79
sister_family_siste

In [17]:
state_abbreviations = {
    'Florida': 'FL',
    'Pennsylvania': 'PA',
    'New York': 'NY',
    'Texas': 'TX',
    'Virginia': 'VA',
    'Michigan': 'MI',
    'Tennessee': 'TN',
    'Mississippi': 'MS',
    'Wisconsin': 'WI',
    'Idaho': 'ID',
    'Georgia': 'GA',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Oklahoma': 'OK',
    'Missouri': 'MO',
    'Ohio': 'OH',
    'Oregon': 'OR',
    'Rhode Island': 'RI',
    'South Dakota': 'SD',
    'Indiana': 'IN',
    'Vermont': 'VT',
    'North Carolina': 'NC',
    'Minnesota': 'MN',
    'Utah': 'UT',
    'Maryland': 'MD',
    'Washington': 'WA',
    'Alaska': 'AK',
    'Illinois': 'IL',
    'Maine': 'ME',
    'South Carolina': 'SC',
    'Arkansas': 'AR',
    'New Jersey': 'NJ',
    'California': 'CA',
    'Colorado': 'CO',
    'Massachusetts': 'MA',
    'North Dakota': 'ND',
    'Wyoming': 'WY'
}

In [18]:
# create a dataframe for each state to check topic representation
for state in state_abbreviations:
    abbreviation = state_abbreviations[state]
    locals()[abbreviation] = school_final[school_final['State'] == state]

In [24]:
TN['Topic'].value_counts()

Topic
school_boy_gay_alex                                107
love_friends_best_school                            82
fantasy_series_war_ash                              45
gender_transgender_lgbt_people                      24
black_white_civil_history                           13
bryce_world_shadowhunters_magnus                    10
keaton_taichi_hiraga_archaeology                     7
killer_theo_police_murder                            6
ancient_gods_mythology_myths                         6
novel_dead_felicity_teeth                            5
vampire_schuyler_bloods_vampires                     5
love_annie_story_classic                             5
art_paintings_works_artists                          5
women_space_science_world                            4
sex_sexual_information_pregnancy                     4
perfect_nishat_companion impulse_cliff               3
memoir_parenthesis_woman_abuse                       3
nazi_story_vonnegut_vonneguts                        3
blac